## Imports

In [2]:
import numpy as np
import tqdm
from tqdm import tqdm_notebook as tqdmnb
import math
import tensorflow as tf
import pandas as pd

## Loading data

In [3]:
text = open('train.txt', 'r')
word_dict = {}
for lines in text:
    line = lines.split()
    for words in line[1:]:
        word_dict.update({line[0] : words})
dictionary = word_dict
reverse_dictionary = dict([(v, k) for k, v in dictionary.items()])

### Баловство

In [25]:
simb2phonDict = {}
gr2coinc = {}
from string import ascii_uppercase
for let in ascii_uppercase:
    gr2coinc.update({let: 0})
phoneme_set = set()
phonemes = reverse_dictionary.keys()
ok = 0
for phoneme in phonemes:
    grapheme = list(reverse_dictionary[phoneme])
    phoneme = phoneme.split('_')
    for p in phoneme:
        phoneme_set.add(p)
    sz = min(len(grapheme), len(phoneme))
    for i in range(sz):
        simb2phonDict.update({grapheme[i]: phoneme[i]})
        if grapheme[i] == phoneme[i]:
            gr2coinc[grapheme[i]] += 1
for g in gr2coinc.keys():
    print(g, "\t", gr2coinc[g], "\t", simb2phonDict[g])
# for g in simb2phonDict.keys():
#     print(g, "\t", simb2phonDict[g])

R 	 20421 	 R
N 	 21124 	 CH
U 	 0 	 N
G 	 6149 	 IY
C 	 0 	 IY
B 	 9752 	 B
E 	 0 	 EH
I 	 0 	 IH
L 	 19260 	 L
Z 	 2062 	 Z
Q 	 0 	 K
M 	 12589 	 AO
F 	 5573 	 F
X 	 0 	 K
K 	 5399 	 K
O 	 0 	 D
Y 	 571 	 S
W 	 2866 	 IH
J 	 0 	 JH
H 	 0 	 HH
A 	 0 	 AO
S 	 16709 	 S
P 	 9572 	 AH
D 	 11822 	 AH
V 	 4890 	 NG
T 	 17937 	 AH


## Preparing data

In [5]:
X_vocab = {}
for let in ascii_uppercase:
    X_vocab.update({let: 0})
X_vocab.update({"'" : 0, '-' : 0})
sz2 = 0
for gr in dictionary.keys():
    for l in gr:
        if l in X_vocab.keys():
            X_vocab[l] += 1
        else:
            X_vocab.update({'UNK': 0})
            print(gr)
    if len(gr) > sz2:
        sz2 = len(gr)


In [6]:
print(X_vocab.items())

dict_items([('R', 47637), ('N', 44939), ('U', 18607), ('G', 16816), ('-', 649), ("'", 4907), ('C', 23850), ('B', 13850), ('E', 69983), ('I', 46686), ('L', 35662), ('Z', 3918), ('Q', 832), ('M', 19204), ('F', 8552), ('X', 1424), ('K', 11002), ('O', 39591), ('Y', 9888), ('W', 6829), ('J', 1662), ('H', 17889), ('A', 54333), ('S', 46389), ('P', 14006), ('D', 21402), ('V', 6467), ('T', 35700)])


In [7]:
import operator
X_sorted = [k for k, v in sorted(X_vocab.items(), key=operator.itemgetter(1))][::-1]

In [8]:
X_let_to_ix = {'ZERO' : 0}
v = 1
for k in X_sorted:
    X_let_to_ix.update({k : v})
    v += 1
X_let_to_ix.update({"UNK" : v})

In [9]:
X_ix_to_let = dict([(v, k) for k, v in X_let_to_ix.items()])

In [10]:
sz1 = len(dictionary.keys())
X_train = np.zeros((sz1, sz2))
i, j = 0, 0
for g in dictionary.keys():
    j = 0
    for l in g:
        X_train[i][j] = X_let_to_ix[l]
        j += 1
    i += 1

In [11]:
print(X)

NameError: name 'X' is not defined

In [13]:
print(len(dictionary.keys()))

83194


In [28]:
print(len(phoneme_set))

39


R
G
B
P
_
S
T
M
K
A
W
H
D
V
N
U
C
E
I
L
Z
O
Y
J
F
